In [1]:
import os
import json
import random
# Use default model (en_core_web_md):
from negate import Negator
# Use a Transformer model with GPU (if available):
negator = Negator(use_transformers=True)

#For GPT-3
from openai import OpenAI
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [2]:
# call the model, might change in the future based on what openAI api updates.
def call_gpt3(prompt_text):
    response = client.completions.create(
    model="text-davinci-003",
    prompt= prompt_text,
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )

    return response.choices[0].text

In [75]:
# the data is in the data folder, link as shown below
data_dir = "data/data_instances_several.json" # the link
f = open(data_dir)
data = json.load(f)
inputList = []
# load the prompt materials (context, questions, answers) in inputList
for datapoint in data["data_samples"]:    
    for i in range(len(datapoint["qa_pairs"])):
        inputList.append({
            "context": datapoint["context"],
            "question": datapoint["qa_pairs"][i]["question"],
            "answer": datapoint["qa_pairs"][i]["answer"]
        })


In [63]:
print(len(inputList))

600


In [32]:
# create the start of the prompt, I use the 2nd one to save cost
def starter_prompt():
    starterPromptList = [
        "Given context contains rules of logical reasoning in natural language. Answer the given question based on context ONLY in ‘yes’ or ‘no’ using logical reasoning ability. DO NOT generate anything as an answer apart from ‘yes’ and ‘no’.",
        "Given the context and question, respond in ‘yes’ or ‘no’.",
        "Answer the given question ONLY in ‘yes’ or ‘no’ using logical reasoning ability. DO NOT generate anything as an answer apart from ’yes’ and ’no’."        
    ]
    dice = random.randint(0, len(starterPromptList))
    # return starterPromptList[dice]
    return starterPromptList[1]

In [12]:
# this block is just for testing purposes, ignore
context = "If someone saves all their money, then they can go on a vacation. If they can go on a vacation, then they will have a great time."
question = "if alex saves all their money, does that mean they will have a great time on vacation?"

prompt = "Given the context that contains rules of logical reasoning in natural language and question, "
prompt += "perform step-by-step reasoning to answer the question. Based on context and reasoning steps "
prompt += "answer the question ONLY in ‘yes’ or ‘no’. Please use the below format: \n"
prompt += f"Context: {context}\n"
prompt += f"Question: {question}\n"
# prompt += "Reasoning steps: [generate step-by-step reasoning]\n"
# prompt += "Answer: Yes/No"

In [30]:
# generate the prompt
def promptGen(data):
    context = data["context"]
    question = data["question"]
    prompt = starter_prompt() + "\n"
    prompt += f"Context: {context}" + "\n"
    prompt += f"Question: {question}"
    return prompt

In [43]:
# make the text lowercase, with no "." and line breaks, basically make them only be "yes" or "no"
def extractText(input):
    result = input.replace('\n','')
    result = result.replace('.','')
    result = result.lower()
    return result

In [76]:
result = {
    "yes": 0,
    "no": 0,
    "yesTrue": 0,
    "noTrue": 0,
}
# the main stuff
for input in inputList:
    prompt = promptGen(input)
    output = call_gpt3(prompt)
    output = extractText(output)
    if input["answer"] == "yes":
        result["yes"]+=1
        if output == input["answer"]:
            result["yesTrue"]+=1
    elif input["answer"] == "no":
        result["no"]+=1
        if output == input["answer"]:
            result["noTrue"]+=1
    else:
        print(output)
    # break




INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST htt

In [77]:
print(result)

{'yes': 150, 'no': 450, 'yesTrue': 1, 'noTrue': 381}


In [80]:
stuff = [{'yes': 150, 'no': 150, 'yesTrue': 74, 'noTrue': 147}]
stuff.append({'yes': 150, 'no': 150, 'yesTrue': 101, 'noTrue': 144})
stuff.append({'yes': 150, 'no': 150, 'yesTrue': 50, 'noTrue': 146})
stuff.append({'yes': 150, 'no': 450, 'yesTrue': 1, 'noTrue': 381})
stuff.append({'yes': 150, 'no': 450, 'yesTrue': 5, 'noTrue': 438})
stuff.append({'yes': 150, 'no': 150, 'yesTrue': 0, 'noTrue': 150})
stuff.append({'yes': 150, 'no': 150, 'yesTrue': 57, 'noTrue': 143})
stuff.append({'yes': 300, 'no': 300, 'yesTrue': 63, 'noTrue': 298})

In [82]:
yesT = 0
yes = 0
noT = 0
no = 0
for data in stuff:
    yes += data['yes']
    no += data['no']
    yesT += data["yesTrue"]
    noT += data["noTrue"]
    

In [84]:
print(yesT/yes, noT/no)

0.26 0.9471794871794872
